# Задание 1.2 - Линейный классификатор (Linear classifier)

В этом задании мы реализуем другую модель машинного обучения - линейный классификатор. Линейный классификатор подбирает для каждого класса веса, на которые нужно умножить значение каждого признака и потом сложить вместе.
Тот класс, у которого эта сумма больше, и является предсказанием модели.

В этом задании вы:
- потренируетесь считать градиенты различных многомерных функций
- реализуете подсчет градиентов через линейную модель и функцию потерь softmax
- реализуете процесс тренировки линейного классификатора
- подберете параметры тренировки на практике

На всякий случай, еще раз ссылка на туториал по numpy:  
http://cs231n.github.io/python-numpy-tutorial/

In [2]:
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

%load_ext autoreload
%autoreload 2

In [3]:
from dataset import load_svhn, random_split_train_val
from gradient_check import check_gradient
from metrics import multiclass_accuracy
import linear_classifer

# Как всегда, первым делом загружаем данные

Мы будем использовать все тот же SVHN.

In [4]:
def prepare_for_linear_classifier(train_X, test_X):
    train_flat = train_X.reshape(train_X.shape[0], -1).astype(np.float32) / 255.0
    test_flat = test_X.reshape(test_X.shape[0], -1).astype(np.float32) / 255.0
    
    
#     вычесть среднее
    mean_image = np.mean(train_flat, axis = 0)
    train_flat -= mean_image
    test_flat -= mean_image
    
    train_flat_with_ones = np.hstack([train_flat, np.ones((train_X.shape[0], 1))])
    test_flat_with_ones = np.hstack([test_flat, np.ones((test_X.shape[0], 1))])
    
    
    return train_flat_with_ones, test_flat_with_ones
    
train_X, train_y, test_X, test_y = \
load_svhn("/home/artem/Загрузки/dlcourse_ai-master/assignments/assignment1/data", max_train=10000, max_test=1000)
train_X, test_X = prepare_for_linear_classifier(train_X, test_X)

# Разделить train на train и val
train_X, train_y, val_X, val_y = random_split_train_val(train_X, train_y, num_val = 1000)

# Играемся с градиентами!

В этом курсе мы будем писать много функций, которые вычисляют градиенты аналитическим методом.

Все функции, в которых мы будем вычислять градиенты, будут написаны по одной и той же схеме.  
Они будут получать на вход точку, где нужно вычислить значение и градиент функции, а на выходе будут выдавать кортеж (tuple) из двух значений - собственно значения функции в этой точке (всегда одно число) и аналитического значения градиента в той же точке (той же размерности, что и вход).
```
def f(x):
    """
    Computes function and analytic gradient at x
    
    x: np array of float, input to the function
    
    Returns:
    value: float, value of the function 
    grad: np array of float, same shape as x
    """
    ...
    
    return value, grad
```

Необходимым инструментом во время реализации кода, вычисляющего градиенты, является функция его проверки. Эта функция вычисляет градиент численным методом и сверяет результат с градиентом, вычисленным аналитическим методом.

Мы начнем с того, чтобы реализовать вычисление численного градиента (numeric gradient) в функции `check_gradient` в `gradient_check.py`. Эта функция будет принимать на вход функции формата, заданного выше, использовать значение `value` для вычисления численного градиента и сравнит его с аналитическим - они должны сходиться.

Напишите часть функции, которая вычисляет градиент с помощью численной производной для каждой координаты. Для вычисления производной используйте так называемую two-point formula (https://en.wikipedia.org/wiki/Numerical_differentiation):

![image](https://wikimedia.org/api/rest_v1/media/math/render/svg/22fc2c0a66c63560a349604f8b6b39221566236d)

Все функции приведенные в следующей клетке должны проходить gradient check.

In [10]:
# TODO: Implement check_gradient function in gradient_check.py
# All the functions below should pass the gradient check

def square(x):
    return float(x*x), 2*x

check_gradient(square, np.array([3.0], np.float32))

def array_sum(x):
    assert x.shape == (2,), x.shape
    return np.sum(x), np.ones_like(x)

check_gradient(array_sum, np.array([3.0, 2.0], np.float32))

def array_2d_sum(x):
    assert x.shape == (2,2)
    return np.sum(x), np.ones_like(x)

check_gradient(array_2d_sum, np.array([[3.0, 2.0], [1.0, 0.0]], np.float32))


------------------------------------
индекс
(0,)
аналитический градиент
6.0
численный градиент
6.008148193359374
------------------------------------
Gradient check passed!
------------------------------------
индекс
(0,)
аналитический градиент
1.0
численный градиент
1.0013580322265625
------------------------------------
------------------------------------
индекс
(1,)
аналитический градиент
1.0
численный градиент
1.0013580322265625
------------------------------------
Gradient check passed!
------------------------------------
индекс
(0, 0)
аналитический градиент
1.0
численный градиент
1.0013580322265625
------------------------------------
------------------------------------
индекс
(0, 1)
аналитический градиент
1.0
численный градиент
1.0013580322265625
------------------------------------
------------------------------------
индекс
(1, 0)
аналитический градиент
1.0
численный градиент
1.0013580322265625
------------------------------------
------------------------------------
индекс

True

## Начинаем писать свои функции, считающие аналитический градиент

Теперь реализуем функцию softmax, которая получает на вход оценки для каждого класса и преобразует их в вероятности от 0 до 1:
![image](https://wikimedia.org/api/rest_v1/media/math/render/svg/e348290cf48ddbb6e9a6ef4e39363568b67c09d3)

**Важно:** Практический аспект вычисления этой функции заключается в том, что в ней учавствует вычисление экспоненты от потенциально очень больших чисел - это может привести к очень большим значениям в числителе и знаменателе за пределами диапазона float.

К счастью, у этой проблемы есть простое решение -- перед вычислением softmax вычесть из всех оценок максимальное значение среди всех оценок:
```
predictions -= np.max(predictions)
```
(подробнее здесь - http://cs231n.github.io/linear-classify/#softmax, секция `Practical issues: Numeric stability`)

In [11]:
# TODO Implement softmax and cross-entropy for single sample
# probs = linear_classifer.softmax(np.array([-10, 0, 10]))
# probs = linear_classifer.softmax(np.array([[-10, 0, 10],\
#                                           [-10, 30, 20]]))
# 
probs = linear_classifer.softmax(np.array([[-10, 0, 10],\
                                          [-10, 30, 20],\
                                          [-50, 60, 20]]))

# print(probs)
# Make sure it works for big numbers too!
probs = linear_classifer.softmax(np.array([1000, 0, 0]))
assert np.isclose(probs[0], 1.0)

Кроме этого, мы реализуем cross-entropy loss, которую мы будем использовать как функцию ошибки (error function).
В общем виде cross-entropy определена следующим образом:
![image](https://wikimedia.org/api/rest_v1/media/math/render/svg/0cb6da032ab424eefdca0884cd4113fe578f4293)

где x - все классы, p(x) - истинная вероятность принадлежности сэмпла классу x, а q(x) - вероятность принадлежности классу x, предсказанная моделью.  
В нашем случае сэмпл принадлежит только одному классу, индекс которого передается функции. Для него p(x) равна 1, а для остальных классов - 0. 

Это позволяет реализовать функцию проще!

In [12]:
# probs = linear_classifer.softmax(np.array([-5, 0, 5]))
# probs = linear_classifer.softmax(np.array([[-5, 0, 5],\
#                                            [-10, 1, 2]]))
probs = linear_classifer.softmax(np.array([[-5, 0, 5],\
                                          [-10, 1, 2],\
                                          [-20, 0, 20]]))

# print(probs)
# print()
# target_index = np.array([[0],[1]])
target_index = np.array([[0],[1],[2]])
linear_classifer.cross_entropy_loss(probs, target_index)


array([1.00067604e+01, 1.31326618e+00, 2.06115369e-09])

После того как мы реализовали сами функции, мы можем реализовать градиент.

Оказывается, что вычисление градиента становится гораздо проще, если объединить эти функции в одну, которая сначала вычисляет вероятности через softmax, а потом использует их для вычисления функции ошибки через cross-entropy loss.

Эта функция `softmax_with_cross_entropy` будет возвращает и значение ошибки, и градиент по входным параметрам. Мы проверим корректность реализации с помощью `check_gradient`.

In [13]:
# TODO Implement combined function or softmax and cross entropy and produces gradient
loss, grad = linear_classifer.softmax_with_cross_entropy(np.array([1, 0, 0]), 0)
check_gradient(lambda x: linear_classifer.softmax_with_cross_entropy(x, 0), np.array([1, 0, 0], np.float32))

------------------------------------
индекс
(0,)
аналитический градиент
-0.42388308
численный градиент
-0.4172325134277343
------------------------------------
------------------------------------
индекс
(1,)
аналитический градиент
0.21194156
численный градиент
0.20861625671386716
------------------------------------
------------------------------------
индекс
(2,)
аналитический градиент
0.21194156
численный градиент
0.20861625671386716
------------------------------------
Gradient check passed!


True

In [14]:
# TODO Implement combined function or softmax and cross entropy and produces gradient
# loss, grad = \
# linear_classifer.softmax_with_cross_entropy(np.array([[1, 0, 0],\
#                                                     [-10, 0, 10]]),\
#                                             np.array([[0],[1]]))
loss, grad = \
linear_classifer.softmax_with_cross_entropy(np.array([[1, 0, 0],\
                                                    [-10, 0, 10],
                                                     [-10, 0, 10]]),\
                                            np.array([[0],[1],[1]]))
print(grad)
# check_gradient(lambda x: \
# linear_classifer.softmax_with_cross_entropy(x,\
#                 np.array([[1],[0],[2]])),\
#                np.array([[1, 0, 0],\
#                          [-10, 0, 10],\
#                          [0,0,1]], np.float32))

[[-4.23883115e-01  2.11941558e-01  2.11941558e-01]
 [ 2.06106005e-09 -9.99954602e-01  9.99954600e-01]
 [ 2.06106005e-09 -9.99954602e-01  9.99954600e-01]]


В качестве метода тренировки мы будем использовать стохастический градиентный спуск (stochastic gradient descent или SGD), который работает с батчами сэмплов. 

Поэтому все наши фукнции будут получать не один пример, а батч, то есть входом будет не вектор из `num_classes` оценок, а матрица размерности `batch_size, num_classes`. Индекс примера в батче всегда будет первым измерением.

Следующий шаг - переписать наши функции так, чтобы они поддерживали батчи.

Финальное значение функции ошибки должно остаться числом, и оно равно среднему значению ошибки среди всех примеров в батче.

In [16]:
# TODO Extend combined function so it can receive a 2d array with batch of samples
np.random.seed(42)
# Test batch_size = 1
num_classes = 5
batch_size = 1
predictions = np.random.randint(-1, 3, size=(batch_size, num_classes)).astype(np.float32)
target_index = np.random.randint(0, num_classes, size=(batch_size, 1)).astype(np.int32)

# print(target_index)
# check_gradient(lambda x: linear_classifer.softmax_with_cross_entropy(x, target_index), predictions)


# linear_classifer.softmax_with_cross_entropy(predictions, target_index)

# Test batch_size = 3
num_classes = 5

batch_size = 5
predictions = np.random.randint(-1, 3, size=(batch_size, num_classes)).astype(np.float32)
target_index = np.random.randint(0, num_classes, size=(batch_size, 1)).astype(np.int32)


check_gradient(lambda x: linear_classifer.softmax_with_cross_entropy(x, target_index), predictions)

# Make sure maximum subtraction for numberic stability is done separately for every sample in the batch
probs = linear_classifer.softmax(np.array([[20,0,0], [1000, 0, 0]]))
assert np.all(np.isclose(probs[:, 0], 1.0))

------------------------------------
индекс
(0, 0)
аналитический градиент
0.038631737
численный градиент
[0.04172325 0.         0.         0.         0.        ]
------------------------------------
------------------------------------
индекс
(0, 1)
аналитический градиент
0.2854521
численный градиент
[0.28014183 0.         0.         0.         0.        ]
------------------------------------
------------------------------------
индекс
(0, 2)
аналитический градиент
0.10501195
численный градиент
[0.10728836 0.         0.         0.         0.        ]
------------------------------------
------------------------------------
индекс
(0, 3)
аналитический градиент
-0.7145479
численный градиент
[-0.7212162  0.         0.         0.         0.       ]
------------------------------------
------------------------------------
индекс
(0, 4)
аналитический градиент
0.2854521
численный градиент
[0.28014183 0.         0.         0.         0.        ]
------------------------------------
-----------

### Наконец, реализуем сам линейный классификатор!

softmax и cross-entropy получают на вход оценки, которые выдает линейный классификатор.

Он делает это очень просто: для каждого класса есть набор весов, на которые надо умножить пиксели картинки и сложить. Получившееся число и является оценкой класса, идущей на вход softmax.

Таким образом, линейный классификатор можно представить как умножение вектора с пикселями на матрицу W размера `num_features, num_classes`. Такой подход легко расширяется на случай батча векторов с пикселями X размера `batch_size, num_features`:

`predictions = X * W`, где `*` - матричное умножение.

Реализуйте функцию подсчета линейного классификатора и градиентов по весам `linear_softmax` в файле `linear_classifer.py`

In [112]:
# TODO Implement linear_softmax function that uses softmax with cross-entropy for linear classifier
batch_size = 2
num_classes = 4
num_features = 3
np.random.seed(42)
W = np.random.randint(-1, 3, size=(num_features, num_classes)).astype(np.float32)
X = np.random.randint(-1, 3, size=(batch_size, num_features)).astype(np.float32)
# target_index = np.ones(batch_size, dtype=np.int32)
target_index = np.random.randint(0, num_classes, size=(batch_size, 1)).astype(np.int32)

# target_index = np.array([[3],[2]])
# print(W)
print(X)
print()
print(target_index)
print()
# print(target_index)
loss, dW = linear_classifer.linear_softmax(X, W, target_index)
print(dW)
check_gradient(lambda w: linear_classifer.linear_softmax(X, w, target_index), W)

[[ 1.  1.  2.]
 [-1.  2.  2.]]

[[3]
 [2]]

[[-0.09645258  0.11127267  0.45977882 -0.47459885]
 [ 0.8893969   0.47394642 -0.9068009  -0.4565424 ]
 [ 1.1215608   0.70611036 -0.90254873 -0.92512244]]
------------------------------------
индекс
(0, 0)
аналитический градиент
-0.09645258
численный градиент
-0.09536743164062499
------------------------------------
------------------------------------
индекс
(0, 1)
аналитический градиент
0.11127267
численный градиент
0.11920928955078124
------------------------------------
------------------------------------
индекс
(0, 2)
аналитический градиент
0.45977882
численный градиент
0.47683715820312494
------------------------------------
------------------------------------
индекс
(0, 3)
аналитический градиент
-0.47459885
численный градиент
-0.47683715820312494
------------------------------------
------------------------------------
индекс
(1, 0)
аналитический градиент
0.8893969
численный градиент
0.9059906005859374
--------------------------------

True

### И теперь регуляризация

Мы будем использовать L2 regularization для весов как часть общей функции ошибки.

Напомним, L2 regularization определяется как

l2_reg_loss = regularization_strength * sum<sub>ij</sub> W[i, j]<sup>2</sup>

Реализуйте функцию для его вычисления и вычисления соотвествующих градиентов.

In [ ]:
# TODO Implement l2_regularization function that implements loss for L2 regularization
linear_classifer.l2_regularization(W, 0.01)
check_gradient(lambda w: linear_classifer.l2_regularization(w, 0.01), W)

# Тренировка!

Градиенты в порядке, реализуем процесс тренировки!

In [116]:
# TODO: Implement LinearSoftmaxClassifier.fit function
classifier = linear_classifer.LinearSoftmaxClassifier()
loss_history = classifier.fit(train_X, train_y, epochs=1, learning_rate=1e-3, batch_size=300, reg=1e1)
# %timeit classifier.\
# fit(train_X, train_y, epochs=1, learning_rate=1e-3, batch_size=300, reg=1e1)


Epoch 0, loss: 2.397227


In [117]:
loss_history

[2.3972274452353637]

In [ ]:
# let's look at the loss history!
plt.plot(loss_history)

In [118]:
# Let's check how it performs on validation set
pred = classifier.predict(val_X)
accuracy = multiclass_accuracy(pred, val_y)
print("Accuracy: ", accuracy)
# val_y

Accuracy:  0.12


In [119]:
# Now, let's train more and see if it performs better

history = classifier.fit(train_X, train_y, epochs=100, learning_rate=1e-3, batch_size=300, reg=1e1)

pred = classifier.predict(val_X)
accuracy = multiclass_accuracy(pred, val_y)
print("Accuracy after training for 100 epochs: ", accuracy)

Epoch 0, loss: 2.330770
Epoch 1, loss: 2.310239
Epoch 2, loss: 2.304199
Epoch 3, loss: 2.302368
Epoch 4, loss: 2.303314
Epoch 5, loss: 2.301739
Epoch 6, loss: 2.301701
Epoch 7, loss: 2.301045
Epoch 8, loss: 2.302496
Epoch 9, loss: 2.301932
Epoch 10, loss: 2.302237
Epoch 11, loss: 2.301880
Epoch 12, loss: 2.302315
Epoch 13, loss: 2.301506
Epoch 14, loss: 2.302030
Epoch 15, loss: 2.301885
Epoch 16, loss: 2.301752
Epoch 17, loss: 2.302007
Epoch 18, loss: 2.302629
Epoch 19, loss: 2.302016
Epoch 20, loss: 2.301907
Epoch 21, loss: 2.302448
Epoch 22, loss: 2.301872
Epoch 23, loss: 2.301326
Epoch 24, loss: 2.301474
Epoch 25, loss: 2.301020
Epoch 26, loss: 2.301005
Epoch 27, loss: 2.301575
Epoch 28, loss: 2.302816
Epoch 29, loss: 2.301255
Epoch 30, loss: 2.302457
Epoch 31, loss: 2.302777
Epoch 32, loss: 2.302266
Epoch 33, loss: 2.301628
Epoch 34, loss: 2.302196
Epoch 35, loss: 2.301847
Epoch 36, loss: 2.302112
Epoch 37, loss: 2.301593
Epoch 38, loss: 2.302066
Epoch 39, loss: 2.301307
Epoch 40, 

### Как и раньше, используем кросс-валидацию для подбора гиперпараметтов.

В этот раз, чтобы тренировка занимала разумное время, мы будем использовать только одно разделение на тренировочные (training) и проверочные (validation) данные.

Теперь нам нужно подобрать не один, а два гиперпараметра! Не ограничивайте себя изначальными значениями в коде.  
Добейтесь точности более чем **20%** на проверочных данных (validation data).

In [128]:
num_epochs = 200
batch_size = 300

learning_rates = [1e-1, 1e-2, 1e-3]
reg_strengths = [1e-5, 1e-6]

best_classifier = None
best_val_accuracy = 0

for l_r in learning_rates:
    for r_s in reg_strengths:
        classifier = linear_classifer.LinearSoftmaxClassifier()
                
        classifier.fit(train_X, train_y, epochs=num_epochs, learning_rate=l_r, batch_size=batch_size, reg=r_s)

        pred = classifier.predict(val_X)
        accuracy = multiclass_accuracy(pred, val_y)
        print('learning_rate: %f' % l_r)
        print('reg_strength: %f' % r_s)
        print('accuracy')
        print(accuracy)

        if best_val_accuracy < accuracy:
            
            best_val_accuracy = accuracy
            print('best validation accuracy achieved: %f' % best_val_accuracy)
            best_classifier = classifier

# TODO use validation set to find the best hyperparameters
# hint: for best results, you might need to try more values for learning rate and regularization strength 
# than provided initially



learning_rate: 0.100000
reg_strength: 0.000010
accuracy
0.247
best validation accuracy achieved: 0.247000
learning_rate: 0.100000
reg_strength: 0.000001
accuracy
0.246
learning_rate: 0.010000
reg_strength: 0.000010
accuracy
0.245
learning_rate: 0.010000
reg_strength: 0.000001
accuracy
0.244
learning_rate: 0.001000
reg_strength: 0.000010
accuracy
0.226
learning_rate: 0.001000
reg_strength: 0.000001
accuracy
0.227


# Какой же точности мы добились на тестовых данных?

In [126]:
test_pred = best_classifier.predict(test_X)
test_accuracy = multiclass_accuracy(test_pred, test_y)
print('Linear softmax classifier test set accuracy: %f' % (test_accuracy, ))

Linear softmax classifier test set accuracy: 0.193000
